# next_pass: predicts satellite overpasses and retrieves available OPERA products 

## Background
* Knowing the time of a satellite overpass (OP) at a precise location is crucial to plan and prepare disaster impact studies. 
The script below can be used to retrieve information about the overpasses of the Landsat 8 & 9 and Sentinel 1 & 2 satellites over a  selected location. For Landsat 8 this occurs every 16 days and for Sentinel 2A / 2B this occurs every 10 days.

* The code calls the Python package 'next_pass' located at https://github.com/ehavazli/next_pass. The latter predicts the next overpass of the satellite of interest by scanning the relevant acquisition plans:
	- Landsat acquisition plans (json files) : https://landsat.usgs.gov/sites/default/files/landsat_acq/assets/json/cycles_full.json
	- Sentinel acquisition plans (KML files to import to Google Earth Pro): https://sentinels.copernicus.eu/web/sentinel/copernicus/sentinel-1/acquisition-plans
      

## Tool Description

All what a user needs to provide is the precise location for which he desires to identify the next overpasses. The location can be input as a single point(latitude, longitude), or bounding box (SNEW coordinates) or a location file path (.kml). The script returns the next collect for Sentinel-1 and Sentinel-2 and the next passes, in ascending and descending directions separately, for Landsat-8 and Landsat-9.

- Specify an Area of Interest (AOI) 
- Run find_next_overpass for Sentinel-1 (A and C), Sentinel-2 and the Landsats (8&9) 
- Visualize each of the above predicted overpass
- Retrieve the most recent available OPERA products at the selected AOI. 

The outputs of next_pass can be compared against overpasses of the site you are interested in using the ESA Orbital Prediction and Overpass Tool (OPOT) at https://evdc.esa.int/orbit/ 


## Getting started
To run next_pass with the given location, run all cells in the notebook starting with the "Load packages" cell.

### Load packages and functions

In [ ]:
import os
import sys
from argparse import Namespace
from datetime import datetime, date
from pathlib import Path
import next_pass
from utils.plot_maps import make_overpasses_map, make_opera_granule_drcs_map, make_opera_granule_map
from utils.opera_products import find_print_available_opera_products, export_opera_products
from utils.utils import valid_drcs_datetime

In [ ]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.INFO)

# Remove handlers added by Jupyter / previous runs
root.handlers.clear()

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)

formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
handler.setFormatter(formatter)

root.addHandler(handler)

### Specify location
Start with selecting the location by  specifying the latitude/longitude (single point or bounding box) or pointing the script to the path of a location file (.kml). 

In [ ]:
# AOI coordinates: can use SNEW or kml file
lon_W, lat_S, lon_E, lat_N = (-54.215, -30.766,-50.814, -28.938)
location_file_path ="locations/JamaicaAOI.kml" # Hurricane Melissa location for this example: Jamaica, October 28, 2025

### Specify satellites of interest 
For now, the tool performs overpasses searches for Sentinel 1A, 1C and 2A, and Landsat 8 and 9.

In [ ]:
# Satellites
sat1 = "sentinel-1"
sat2 = "sentinel-2"
sat3 = "landsat"

### Specify number of days for previous overpasses look up
The tool also retrieves the most recent previous overpasses. The user specifies the number of days to look back for past overpasses, otherwise the default is 13 days.

In [ ]:
nb_days = 7

### Decide on prediction/retrieval or not of cloudiness for future/past overpasses
The tool allows users to call the weather API oenp-meteo to display cloudiness prediction and/or history for future and past overpasses, respectively. The default is "cloudiness = false" as this process is computationally expensive compared to other next_pass features (**up to 20 minutes for each satellite**).  

In [ ]:
cloud_info = False 

## Run next_pass
use next_pass to predict the overpasses of the above satellites over the selected location. The cells below will call next_pass to predict overpasses for the selected satellites, then provide overpasses vizualisation in an interactive map.

In [ ]:
# Create the args object to pass to find_next_overpass
args = Namespace(
    bbox=location_file_path,  # if we are using a path to a location kml file
    sat=sat1,  # assuming sat1 is defined (e.g., 'sentinel-1', 'sentinel-2', 'landsat')
    look_back=nb_days,  # number of days to look back for past overpasses
    cloudiness = cloud_info # for cloudiness prediction and/or history
)

# Create the output directory to save the generated maps later in this notebook
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") 
timestamp_dir = Path(f"nextpass_outputs_{timestamp}")
timestamp_dir.mkdir(parents=True, exist_ok=True)


In [ ]:
print("*** ",sat1," ***")
logging.disable(logging.CRITICAL)
result1 = next_pass.find_next_overpass(args)
logging.disable(logging.NOTSET)
s1_results = result1["sentinel-1"]
s1_next_collect_info = s1_results.get("next_collect_info", "No collection info available")
s1_next_collect_geometry = s1_results.get("next_collect_geometry", None)
print(s1_next_collect_info)

In [ ]:
print("*** ",sat2," ***")
args.sat = sat2
logging.disable(logging.CRITICAL)
result2 = next_pass.find_next_overpass(args)
logging.disable(logging.NOTSET)
s2_results = result2["sentinel-2"]
s2_next_collect_info = s2_results.get("next_collect_info", "No collection info available")
s2_next_collect_geometry = s2_results.get("next_collect_geometry", None)
print(s2_next_collect_info)

In [ ]:
print("*** ",sat3," ***")
args.sat = sat3
logging.disable(logging.CRITICAL)
result3 = next_pass.find_next_overpass(args)
logging.disable(logging.NOTSET)
l_results = result3["landsat"]
l_next_collect_info = l_results.get("next_collect_info", "No collection info available")
l_next_collect_geometry = l_results.get("next_collect_geometry", None)
print(l_next_collect_info)

### Overpasses Visualisation  
The next_pass overpasses visualization tool shows the path of a selected satellite at the predicted date/time. It creates and saves an HTML satellite overpasses map in the inputted run folder.  

In [ ]:
overpasses_map = make_overpasses_map(s1_results, s2_results, l_results, args.bbox, timestamp_dir)
overpasses_map

# Available OPERA Products

## Description
* Here we will use Leafmap and Earthaccess to explore OPERA DSWx Products.
* The Leafmap library provides a suite of tools for interactive mapping and visualization in Jupyter Notebooks. Leafmap version 0.30.0 and later offers tools specifically for accessing NASA Earthdata by building on the newly developed NASA Earthaccess library. Earthaccess provides streamlined access to NASA Earthdata and simplifies the authentication and querying process over previously developed approaches. 
* This section is designed to leverage tools within Earthaccess and Leafmap to facilitate easier access and visualization of OPERA data products for a user-specified area of interest (AOI).
* By default, he next_pass component for OPERA products search will look for a predefined list of opera datasets, namely opera_datasets = [OPERA_L3_DSWX-HLS_V1, OPERA_L3_DSWX-S1_V1, OPERA_L3_DIST-ALERT-HLS_V1, OPERA_L3_DIST-ANN-HLS_V1, OPERA_L2_RTC-S1_V1, OPERA_L2_CSLC-S1_V1, OPERA_L3_DISP-S1_V1]. If the user is interested in a reduced set of OPERA products, he can run next_pass with the option -p (--products) and a list containing a subset of OPERA products to be searched. 

## OPERA DSWx Products 

The Dynamic Surface Water eXtent (DSWx) products map pixel-wise surface water detections using optical or SAR imagery. The DSWx suite is composed of complementary products, which are named according to their input datasets. Later in the present section, we will use the -p option to focus on: 

- DSWx from Harmonized Landsat Sentinel-2 (DSWx-HLS)
- DSWx from Sentinel-1 (DSWx-S1)


## Display most recent OPERA products at the selected Area of Interest (AOI)
The script will use the area of interest indicated above (to predict overpasses) to retrieve the five most recent OPERA products at the time the present notebook is run. next_pass will export the obtained results to an "opera_products_metadata.xlsx" file and visualise the OPERA granules in an "opera_products_map.html" map HTML file, both saved in the output folder created earlier.  

In [ ]:
number_of_dates = 5        # Number of most recent dates to consider for OPERA products
event_date = "today"  # can be today or a previous date (UTC) in format YYYY-MM-DD to consider for OPERA products
list_of_products = ["DSWX-HLS_V1","DSWX-S1_V1"]  # sublist of products to consider for OPERA products search

results_opera = find_print_available_opera_products(args.bbox, number_of_dates, event_date, list_of_products)
export_opera_products(results_opera, timestamp_dir)

In [ ]:
opera_products_map = make_opera_granule_map(results_opera, args.bbox, timestamp_dir)
opera_products_map

## Display availability of OPERA products for a specific event date & time at the selected AOI
Upon request from the user, by calling next_pass with the option -g (--generate-drcs-html) and a UTC event date in format "YYYY-MM-DDTHH:MM", next_pass generates a DRCS OPERA products HTML file. The DRCS map displays post-event OPERA products, with active download links. Pre-event granules with upcoming satellite overpass information are grayed out. The upcoming passes are given in UTC, Operator TZ, and Event Location TZ. 

In [ ]:
event_date_time = "2025-12-10T01:00"
generate_drcs_map = valid_drcs_datetime(event_date_time)
opera_products_drcs_map = make_opera_granule_drcs_map(generate_drcs_map, results_opera, s1_results, s2_results, l_results, args.bbox, 
                                                       timestamp_dir)
opera_products_drcs_map

***

# Examples of next_pass CLI usage
Please restart kernel before running each of the below examples to make sure they run as intended (in terms of output visualisation)

## Example of most basic call with just a bbox to indicate the AOI

In [ ]:
import next_pass
next_pass.run_next_pass([34.15, 34.25, -118.20, -118.15])

## Example of next_pass call with DRCS map request for the Washington flooding AOI
For now, the example uses 2025, Dec 07, 02:00 am as event date. Please feel free to change this to a more recent date and time. 
This call queries overpasses from the previous five days and restricts the search to a selected subset of OPERA products.

In [ ]:
import next_pass
import subprocess
cmd = ["next-pass",
       "-b", "44.75", "49", "-125", "-121",
        "-g", "2025-12-07T02:00",
        "-k", "5",
        "-p", "DSWX-HLS_V1", "DSWX-S1_V1", "DIST-ALERT-HLS_V1", "DIST-ANN-HLS_V1"
      ]
result = subprocess.run(cmd)
if result.returncode == 0:
    print("Success")
elif result.returncode == 1:
    print("Completed with warnings or informational status")
else:
    print("Failure:", result.returncode)